# Example calculation of PV potential for ERA5 data

In [ ]:
import xarray as xr
import numpy as np
import time
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
from glob import glob 

In [ ]:
# location of era5 data on teachinghub
path="/home/yannickh00/LEHRE/msc-intro-comp-met-ex-w2024/data/era5/"

In [ ]:
cluster = LocalCluster(dashboard_adress=":8788"
client = Client(cluster, n_workers=20, threads_per_worker=5)

In [ ]:
year = '2000'
filenames = glob(path+'era5-{year}*.nc')
ds=xr.open_mfdataset(filenames)
print(filenames)

Calculate wind speed.

In [ ]:
ds["wspd"] = np.sqrt(np.power(ds["u10"],2)+np.power(ds["v10"],2))

Calculate PV potential following Jerez et al. 2015, https://www.nature.com/articles/ncomms10014.

**Note:** radiative fluxes are accumulated over 1 hour, so we need to divided by seconds per hour to obtain fluxes in Wm-2.

In [5]:
def calculate_pv_pot(ds): 
    sechour=3600 # secondd per hour

    c1 = 4.3
    c2 = 0.943
    c3 = 0.028
    c4 = -1.528
    
    # cell temperature
    T_cell = c1 + c2 * (ds.t2m - 273.15) + c3 * ds.ssrd/sechour + c4 * ds.wspd
    
    # performance ratio
    beta = -0.005
    p_r = 1 + beta*(T_cell-25)
    
    # pv potential
    pv_pot = p_r * ds.ssrd/(sechour) * 1/1000
    return pv_pot

Calculate time-mean PV potential and plot as a map.

In [ ]:
pv_pot = calculate_pv_pot(ds)
pv_pot_tmean = pv_pot.mean("valid_time").compute()

In [ ]:
plt.contourf(ds.longitude, ds.latitude, pv_pot_tmean)
plt.colorbar()
plt.safefig('5_year_mean.png')

In [ ]:
# 